<a href="https://colab.research.google.com/github/WizardOfCodes442/student-facial-biometric-project/blob/main/modeltestmd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {
    "id": "intro"
   },
   "source": [
    "# Facial Recognition CNN Model Testing\n",
    "\n",
    "This notebook demonstrates the implementation and testing of our CNN model for facial recognition in a student examination environment.\n",
    "\n",
    "## Setup and Dependencies"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "id": "setup"
   },
   "source": [
    "!pip install tensorflow opencv-python-headless scikit-learn pandas matplotlib\n",
    "\n",
    "import tensorflow as tf\n",
    "from tensorflow.keras.applications import MobileNetV2\n",
    "from tensorflow.keras.applications.mobilenet_v2 import preprocess_input\n",
    "from tensorflow.keras import layers, models\n",
    "import numpy as np\n",
    "import cv2\n",
    "import matplotlib.pyplot as plt\n",
    "from sklearn.model_selection import train_test_split\n",
    "import os\n",
    "from google.colab import files\n",
    "\n",
    "# Check if GPU is available\n",
    "print(\"Num GPUs Available: \", len(tf.config.list_physical_devices('GPU')))"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {
    "id": "model-architecture"
   },
   "source": [
    "## Model Architecture\n",
    "\n",
    "We'll implement the CNN model as described in the original architecture."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "id": "cnn-model"
   },
   "source": [
    "INPUT_SHAPE = (224, 224, 3)\n",
    "\n",
    "def create_model(num_classes):\n",
    "    base_model = MobileNetV2(\n",
    "        input_shape=INPUT_SHAPE,\n",
    "        include_top=False,\n",
    "        weights='imagenet'\n",
    "    )\n",
    "    \n    for layer in base_model.layers:\n",
    "        layer.trainable = False\n",
    "    
    "    model = models.Sequential([\n",
    "        base_model,\n",
    "        layers.GlobalAveragePooling2D(),\n",
    "        layers.Dense(256, activation='relu'),\n",
    "        layers.BatchNormalization(),\n",
    "        layers.Dropout(0.5),\n",
    "        layers.Dense(num_classes, activation='softmax')\n",
    "    ])\n",
    "    \n",
    "    return model\n",
    "\n",
    "# Compile model\n",
    "def compile_model(model):\n",
    "    model.compile(\n",
    "        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),\n",
    "        loss='categorical_crossentropy',\n",
    "        metrics=['accuracy']\n",
    "    )\n",
    "    return model"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {
    "id": "data-preparation"
   },
   "source": [
    "## Data Preparation\n",
    "\n",
    "We'll create functions to load and preprocess our dataset."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "id": "data-functions"
   },
   "source": [
    "def load_and_preprocess_image(image_path):\n",
    "    img = cv2.imread(image_path)\n",
    "    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)\n",
    "    \n",
    "    # Face detection\n",
    "    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')\n",
    "    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)\n",
    "    faces = face_cascade.detectMultiScale(gray, 1.1, 4)\n",
    "    \n",
    "    if len(faces) > 0:\n",
    "        x, y, w, h = faces[0]\n",
    "        face = img[y:y+h, x:x+w]\n",
    "    else:\n",
    "        face = img\n",
    "    \n",
    "    face = cv2.resize(face, (INPUT_SHAPE[0], INPUT_SHAPE[1]))\n",
    "    face = preprocess_input(face)\n",
    "    \n",
    "    return face\n",
    "\n",
    "def prepare_dataset(data_dir):\n",
    "    images = []\n",
    "    labels = []\n",
    "    label_mapping = {}\n",
    "    \n",
    "    for idx, student_folder in enumerate(os.listdir(data_dir)):\n",
    "        label_mapping[idx] = student_folder\n",
    "        student_path = os.path.join(data_dir, student_folder)\n",
    "        \n",
    "        if os.path.isdir(student_path):\n",
    "            for image_file in os.listdir(student_path):\n",
    "                image_path = os.path.join(student_path, image_file)\n",
    "                try:\n",
    "                    processed_image = load_and_preprocess_image(image_path)\n",
    "                    images.append(processed_image)\n",
    "                    labels.append(idx)\n",
    "                except Exception as e:\n",
    "                    print(f\"Error processing {image_path}: {e}\")\n",
    "    \n",
    "    return np.array(images), np.array(labels), label_mapping"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {
    "id": "data-upload"
   },
   "source": [
    "## Data Upload and Processing\n",
    "\n",
    "Upload your dataset to Colab. The dataset should be organized with one folder per student, containing their facial images."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "id": "upload-data"
   },
   "source": [
    "# Mount Google Drive if your dataset is there\n",
    "from google.colab import drive\n",
    "drive.mount('/content/drive')\n",
    "\n",
    "# Update this path to where your dataset is located\n",
    "DATA_DIR = '/content/drive/MyDrive/student_dataset'\n",
    "\n",
    "# Prepare dataset\n",
    "X, y, label_mapping = prepare_dataset(DATA_DIR)\n",
    "num_classes = len(label_mapping)\n",
    "\n",
    "# Convert labels to one-hot encoding\n",
    "y_onehot = tf.keras.utils.to_categorical(y, num_classes)\n",
    "\n",
    "# Split dataset\n",
    "X_train, X_test, y_train, y_test = train_test_split(\n",
    "    X, y_onehot, test_size=0.2, random_state=42\n",
    ")\n",
    "\n",
    "print(f\"Training samples: {len(X_train)}\")\n",
    "print(f\"Test samples: {len(X_test)}\")\n",
    "print(f\"Number of classes: {num_classes}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {
    "id": "training"
   },
   "source": [
    "## Model Training"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "id": "train-model"
   },
   "source": [
    "# Create and compile model\n",
    "model = create_model(num_classes)\n",
    "model = compile_model(model)\n",
    "\n",
    "# Train model\n",
    "history = model.fit(\n",
    "    X_train, y_train,\n",
    "    validation_split=0.2,\n",
    "    epochs=20,\n",
    "    batch_size=32\n",
    ")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {
    "id": "evaluation"
   },
   "source": [
    "## Model Evaluation"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "id": "evaluate-model"
   },
   "source": [
    "# Plot training history\n",
    "plt.figure(figsize=(12, 4))\n",
    "\n",
    "plt.subplot(1, 2, 1)\n",
    "plt.plot(history.history['accuracy'], label='Training Accuracy')\n",
    "plt.plot(history.history['val_accuracy'], label='Validation Accuracy')\n",
    "plt.title('Model Accuracy')\n",
    "plt.xlabel('Epoch')\n",
    "plt.ylabel('Accuracy')\n",
    "plt.legend()\n",
    "\n",
    "plt.subplot(1, 2, 2)\n",
    "plt.plot(history.history['loss'], label='Training Loss')\n",
    "plt.plot(history.history['val_loss'], label='Validation Loss')\n",
    "plt.title('Model Loss')\n",
    "plt.xlabel('Epoch')\n",
    "plt.ylabel('Loss')\n",
    "plt.legend()\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()\n",
    "\n",
    "# Evaluate on test set\n",
    "test_loss, test_accuracy = model.evaluate(X_test, y_test)\n",
    "print(f\"Test accuracy: {test_accuracy:.4f}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {
    "id": "testing"
   },
   "source": [
    "## Test Individual Images"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "id": "test-individual"
   },
   "source": [
    "def test_image(image_path):\n",
    "    # Load and preprocess image\n",
    "    processed_image = load_and_preprocess_image(image_path)\n",
    "    \n",
    "    # Make prediction\n",
    "    prediction = model.predict(np.expand_dims(processed_image, axis=0))\n",
    "    predicted_class = np.argmax(prediction[0])\n",
    "    confidence = prediction[0][predicted_class]\n",
    "    \n",
    "    # Display results\n",
    "    plt.figure(figsize=(8, 4))\n",
    "    plt.subplot(1, 2, 1)\n",
    "    plt.imshow(cv2.imread(image_path)[...,::-1])\n",
    "    plt.title('Input Image')\n",
    "    plt.axis('off')\n",
    "    \n",
    "    plt.subplot(1, 2, 2)\n",
    "    plt.imshow(processed_image / 2 + 0.5)  # Denormalize\n",
    "    plt.title(f'Processed Image\\nPredicted: {label_mapping[predicted_class]}\\nConfidence: {confidence:.4f}')\n",
    "    plt.axis('off')\n",
    "    \n",
    "    plt.show()\n",
    "\n",
    "# Test an individual image\n",
    "# You can upload an image using files.upload() or specify a path\n",
    "uploaded = files.upload()\n",
    "for filename in uploaded.keys():\n",
    "    test_image(filename)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {
    "id": "save-model"
   },
   "source": [
    "## Save Model"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "id": "save"
   },
   "source": [
    "# Save model\n",
    "model.save('facial_recognition_model.h5')\n",
    "\n",
    "# Save label mapping\n",
    "import json\n",
    "with open('label_mapping.json', 'w') as f:\n",
    "    json.dump({str(k): v for k, v in label_mapping.items()}, f)\n",
    "\n",
    "# Download files\n",
    "files.download('facial_recognition_model.h5')\n",
    "files.download('label_mapping.json')"
   ]
  }
 ],
 "metadata": {
  "accelerator": "GPU",
  "colab": {
   "private_outputs": true,
   "provenance": []
  },
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.0"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}